# Initial Research:
This notebook is testing the practical applications of replacing earlier, large activation maps in CNNs with gabor-filter features to reduce training time and increase convergence rate.
- This article discusses benefits of use of Gabor filters to supplement CNNs: https://www.mdpi.com/2079-9292/12/19/4072
- Article emphasizing the difficulty of tuning parameters for gabor filters: https://homepages.inf.ed.ac.uk/rbf/CAVIAR/PAPERS/05-ibpria-features.pdf

Findings:
Implementation of gabor filters led to higher accuracy in fewer epochs. Gabor filters with a reduced CNN size had comparable loss and accuracy with significantly reduced training time and fewer paramters.

In [1]:
import tensorflow as tf
import numpy as np
from skimage import color, filters, io

In [2]:
# Load CIFAR-10 dataset
cifar10 = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Convert images to numpy arrays
train_images = np.array(train_images)[:5000]
test_images = np.array(test_images)[:500]
train_labels = train_labels[:5000]
test_labels = test_labels[:500]

170498071/170498071 [==============================] - 12s 0us/step


In [3]:
theta = np.linspace(0,3,5)
omega = np.linspace(1,5,4)
gabor_train_images = []
gabor_test_images = []

for ind, val in enumerate(train_images):
  gray_image = color.rgb2gray(val)
  ph = []
  for th in theta:
    for om in omega:
      ph.append(filters.gabor(gray_image, om, th)[0])
  gabor_train_images.append(np.stack(ph, axis=-1))

for ind, val in enumerate(test_images):
  gray_image = color.rgb2gray(val)
  ph = []
  for th in theta:
    for om in omega:
      ph.append(filters.gabor(gray_image, om, th)[0])
  gabor_test_images.append(np.stack(ph, axis=-1))

gabor_train_images = np.stack(gabor_train_images)
gabor_test_images = np.stack(gabor_test_images)


In [4]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # Assuming 10 classes for classification
])

# Compile the model
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy()])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))


Epoch 1/10
157/157 [==============================] - 8s 40ms/step - loss: 3.7241 - sparse_categorical_accuracy: 0.1888 - val_loss: 2.0960 - val_sparse_categorical_accuracy: 0.2300
Epoch 2/10
157/157 [==============================] - 8s 50ms/step - loss: 1.8961 - sparse_categorical_accuracy: 0.3032 - val_loss: 1.8871 - val_sparse_categorical_accuracy: 0.3340
Epoch 3/10
157/157 [==============================] - 6s 39ms/step - loss: 1.7376 - sparse_categorical_accuracy: 0.3742 - val_loss: 1.7482 - val_sparse_categorical_accuracy: 0.3520
Epoch 4/10
157/157 [==============================] - 8s 48ms/step - loss: 1.5687 - sparse_categorical_accuracy: 0.4254 - val_loss: 1.7092 - val_sparse_categorical_accuracy: 0.4020
Epoch 5/10
157/157 [==============================] - 6s 38ms/step - loss: 1.4442 - sparse_categorical_accuracy: 0.4766 - val_loss: 1.7464 - val_sparse_categorical_accuracy: 0.3980
Epoch 6/10
157/157 [==============================] - 8s 49ms/step - loss: 1.3029 - sparse_cate

In [5]:
gabor_model = models.Sequential([
    #layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,20)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # Assuming 10 classes for classification
])

# Compile the model
gabor_model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy()])

# Train the model
gabor_model.fit(gabor_train_images, train_labels, epochs=10, validation_data=(gabor_test_images, test_labels))


Epoch 1/10
157/157 [==============================] - 4s 23ms/step - loss: 2.2007 - sparse_categorical_accuracy: 0.1904 - val_loss: 2.1207 - val_sparse_categorical_accuracy: 0.2380
Epoch 2/10
157/157 [==============================] - 3s 22ms/step - loss: 1.9163 - sparse_categorical_accuracy: 0.3118 - val_loss: 1.8444 - val_sparse_categorical_accuracy: 0.3320
Epoch 3/10
157/157 [==============================] - 5s 31ms/step - loss: 1.7368 - sparse_categorical_accuracy: 0.3794 - val_loss: 1.7360 - val_sparse_categorical_accuracy: 0.3540
Epoch 4/10
157/157 [==============================] - 3s 22ms/step - loss: 1.5745 - sparse_categorical_accuracy: 0.4466 - val_loss: 1.7561 - val_sparse_categorical_accuracy: 0.3880
Epoch 5/10
157/157 [==============================] - 3s 22ms/step - loss: 1.4585 - sparse_categorical_accuracy: 0.4922 - val_loss: 1.7476 - val_sparse_categorical_accuracy: 0.3920
Epoch 6/10
157/157 [==============================] - 3s 22ms/step - loss: 1.3173 - sparse_cate